In [1]:
!pip install opencv-python mediapipe matplotlib

In [2]:
import os
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import mediapipe as mp
from mediapipe.python.solutions import holistic as mp_holistic

# PyTorch Hub
import torch

In [3]:
# 모델(yolov5) 가져오기
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# 우리는 사람을 구분하는 것에 관심이 있으니깐
yolo_model.classes=[0]

Using cache found in C:\Users\seoeu/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-5-17 Python-3.8.13 torch-1.11.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:
mp_drawing = mp.solutions.drawing_utils
# 원래 우리 코드에서는 mp_holistic을 사용함
# mp_pose를 mp_holistic으로 바꿔서 해봐야겠다
mp_holistic =mp.solutions.holistic

In [5]:
#video_path ="walking1.mp4"

In [6]:
# 비디오의 면적 구하기
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture(video_path)
while cap.isOpened():
    ret, frame = cap.read()
    h, w, _ = frame.shape
    size = (w, h)
    print(size)
    break

(640, 480)


In [8]:
cap = cv2.VideoCapture(0)

#for webacam cv2.VideoCapture(NUM) NUM -> 0,1,2 for primary and secondary webcams..

#For saving the video file as output.avi
# output.avi 생성. 우리는 필요 없음. 나중에 지워도 되는 코드
#out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20, size)
while cap.isOpened():    
    ret, frame = cap.read()

    # BGR에서 RGB로 변경
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # 예측 성능을 높이기 위해선 얘를 False로 하는 게 좋다고 함
    image.flags.writeable = False    

    result = yolo_model(image)    
    
    # 렌더링 위해 이미지를 다시 BGR로 바꿈
    image.flags.writeable = True   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # print(result.xyxy)  # img1 predictions (tensor)

    # 크롭된 이미지들 모아두는 배열(사람 개개인별로 분류한 것들)
    img_list =[]
    
    # 사람을 올바르게 구분하려면 약간의 여유 공간이 있는 게 좋다고 함
    MARGIN=10

    # 이게 사람 구분하는 코드
    for (xmin, ymin, xmax, ymax, confidence, clas) in result.xyxy[0].tolist():
      with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
        #Media pose prediction ,we are 
        results = holistic.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:])

        #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing 
        mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                              ) 
        img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
    cv2.imshow('walking', image)

    # writing in the video file 
    #out.write(image)

    ## Code to quit the video incase you are using the webcam             
    # cv2.imshow('Activity recognition', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
#out.release()
cv2.destroyAllWindows()